This notebook is to split into train and test, make standardization, and build the model, then pickle it.

In [1]:
import pandas as pd
url="C:\\Users\\reemz\\Desktop\\SeniorProject - Copy\\newspmodel\\dataset_all.csv"
dataset=pd.read_csv(url)

In [2]:
# split and standardization
import numpy as np
from sklearn.model_selection import train_test_split
#split train and test
train_set, test_set = train_test_split(dataset, test_size=0.1, random_state=42,stratify=dataset['Label'])
train_set.reset_index(drop=True, inplace=True)
test_set.reset_index(drop=True, inplace=True)
#divide X and y
train_X = train_set.drop("Label", axis=1)
train_y = train_set["Label"].copy()
test_X = test_set.drop("Label", axis=1)
test_y = test_set["Label"].copy()
#data standardization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(train_X)
scaled=scaler.transform(train_X)  #scaled is numby array
train_X = pd.DataFrame(scaled, columns=train_X.columns)
scaled=scaler.transform(test_X)
test_X = pd.DataFrame(scaled, columns=test_X.columns)

In [3]:
scaler.mean_

array([ 1.40311968e-01,  1.37064780e-01,  1.40689393e-01,  1.36070745e-01,
        1.27493198e-01,  1.26311267e-01,  9.06706732e-02,  8.22238996e-02,
        7.04626563e-02,  8.76929812e-02,  9.83149193e-02,  1.03883129e-01,
       -1.67913532e-02, -2.39342341e-02, -4.97761461e-02, -4.52058754e-02,
       -5.95031481e-02, -5.98260161e-02,  9.08457163e-02,  1.09706754e-01,
        7.37393109e-02,  8.23663727e-02,  8.26899317e-02,  7.89591442e-02,
        1.43043124e-02,  3.16472032e-02,  2.36744253e-02,  2.48744237e-02,
        2.34665876e-02,  2.01547407e-02, -2.24116243e-02, -1.65964683e-02,
        4.44428118e-04, -6.43380602e-03, -1.44542368e-02, -1.80471966e-02,
        3.57331399e+03,  3.21099956e+03,  3.14654423e+03,  3.08832015e+03,
        3.30822238e+03,  3.53313478e+03,  3.37468407e+03,  3.07477560e+03,
        2.99862467e+03,  3.03053951e+03,  3.18013941e+03,  3.24686599e+03,
        3.19850745e+03,  2.48550587e+03,  2.51977548e+03,  2.55828525e+03,
        2.80811411e+03,  

In [4]:
scaler.var_

array([9.18142265e-02, 8.56612481e-02, 7.92708316e-02, 7.90201849e-02,
       8.17278830e-02, 8.32505725e-02, 1.70966853e-02, 9.83449067e-03,
       4.45859126e-03, 7.36584275e-03, 1.20640274e-02, 1.67802926e-02,
       8.98897889e-02, 6.34096635e-02, 3.22170435e-02, 3.00310149e-02,
       3.06501264e-02, 3.35385511e-02, 1.60062238e-02, 1.91338476e-02,
       1.77709763e-02, 1.29380331e-02, 1.17504649e-02, 1.41640326e-02,
       3.68169261e-02, 2.09196289e-02, 2.20689477e-02, 2.10810977e-02,
       2.66829485e-02, 3.22278245e-02, 1.15358249e-02, 8.77768903e-03,
       1.86429815e-03, 4.29765945e-03, 7.43272569e-03, 9.76884269e-03,
       8.91711007e+05, 1.62607986e+06, 1.29692174e+06, 1.27932403e+06,
       6.82077296e+05, 2.92340045e+05, 9.83617118e+04, 8.38161760e+05,
       8.56438598e+05, 7.54877685e+05, 4.29868993e+05, 3.74068043e+05,
       6.72824759e+05, 2.52042445e+06, 2.14738553e+06, 2.06229070e+06,
       1.54108138e+06, 1.37034514e+06, 9.00164275e+05, 2.49935767e+06,
      

In [5]:
#building random forest model
import sys
!{sys.executable} -m pip install sklearn-json

import sklearn_json as skljson
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
forest_clf = RandomForestClassifier(random_state=42)
modelj = forest_clf.fit(train_X, train_y)
scores=cross_val_score(forest_clf, train_X, train_y, cv=5, scoring="accuracy")
scores

array([0.96969697, 0.96969697, 0.9375    , 0.90625   , 0.84375   ])

In [6]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

In [7]:
display_scores(scores)

Scores: [0.96969697 0.96969697 0.9375     0.90625    0.84375   ]
Mean: 0.9253787878787879
Standard deviation: 0.047125992401005366


In [8]:
#error analysis
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
train_y_pred = cross_val_predict(forest_clf, train_X, train_y, cv=5)
conf_mx = confusion_matrix(train_y,train_y_pred)
conf_mx

array([[18,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0, 15,  3,  0,  0,  0,  0,  0,  0],
       [ 0,  2, 16,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0, 18,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, 15,  3,  0,  0,  0],
       [ 0,  0,  0,  1,  2, 15,  0,  0,  0],
       [ 1,  0,  0,  0,  0,  0, 17,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0, 18,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0, 18]], dtype=int64)

In [9]:
from sklearn.metrics import precision_score, recall_score, accuracy_score
recall_score(train_y,train_y_pred,average=None)

array([1.        , 0.83333333, 0.88888889, 1.        , 0.83333333,
       0.83333333, 0.94444444, 1.        , 1.        ])

In [10]:
precision_score(train_y,train_y_pred,average=None)

array([0.94736842, 0.88235294, 0.84210526, 0.94736842, 0.88235294,
       0.83333333, 1.        , 1.        , 1.        ])

In [11]:
accuracy_score(train_y,train_y_pred)

0.9259259259259259

1. from the confusion matrix, it is clear it classify sign 1 as sign 6. The total of each sign in tyrain set is 18.
2. from recall, sometimes it cannot capture sign 1 and 4. Recall is the actual percentage of each sign that the model can capture correctly.
3. from precision, when the model say sign 6, the chance of true classification is 0.85. Precision is the percentage of correct classifications among all classifications. 

# test the model

In [12]:
final_predictions = forest_clf.predict(test_X)

from sklearn.metrics import accuracy_score
accuracy_score(test_y, final_predictions)

0.9444444444444444

# pickle the model

In [13]:
from pickle import dump, load
import json
# save the model to disk
modelName = 'C:\\Users\\reemz\\Desktop\\SeniorProject - Copy\\newspmodel\\finalized_model.pkl'
dump(forest_clf, open(modelName, 'wb'))
# save the scaler 
scalerName = 'C:\\Users\\reemz\\Desktop\\SeniorProject - Copy\\newspmodel\\scaler.pkl'
dump(scaler, open(scalerName, 'wb'))


modelJson ='C:\\Users\\reemz\\Desktop\\SeniorProject - Copy\\newspmodel\\modelj.json'
skljson.to_json(modelj, modelJson)
#deserialized_model = skljson.from_json(modelJson)


# explore the performance of the pickled model

In [14]:
#file path XXX.sav
filename = 'C:\\Users\\reemz\\Desktop\\SeniorProject - Copy\\newspmodel\\finalized_model.pkl'
loaded_model = load(open(filename, 'rb'))
scalerName = 'C:\\Users\\reemz\\Desktop\\SeniorProject - Copy\\newspmodel\\scaler.pkl'
scaler2 = load(open(scalerName, 'rb'))

# 88 used to be 72, changed because of an error
scaled_new=scaler2.transform(dataset.iloc[:,0:66])  #scaled is numby array
scaled_new = pd.DataFrame(scaled_new)

results = loaded_model.predict(scaled_new )
accuracy_score(dataset.iloc[:,-1], results)

0.9944444444444445

In [15]:
results

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 6, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8], dtype=int64)

In [16]:
#test using the pickled model
# load the model from disk

#file path XXX.sav
filename = 'C:\\Users\\reemz\\Desktop\\SeniorProject - Copy\\newspmodel\\finalized_model.pkl'
loaded_model = load(open(filename, 'rb'))
s=3

# 88 used to be 72, changed because of an error,, new is 66
dd = np.zeros(shape=(1,66))
dd[0][:]=dataset.iloc[s,0:66]
some_sign=scaler2.transform(dd)  #scaled is numby array
#some_sign = pd.DataFrame(some_sign)
result = loaded_model.predict(some_sign)
print("The predicted sign is: ",result)
print("The predicted sign is: ",result[0],"\nThe actual sign is: ",dataset.iloc[s,-1])

The predicted sign is:  [0]
The predicted sign is:  0 
The actual sign is:  0
